Problem: Maximize the number of possible team ups, with constraints:
- A team consists of 6 characters
- maximize win rate while every hero is in a team up


team ups are powerful, but their strength is hard to quantify, therefore not very useful for solving the main problem: In this subproblem, we explore the ideal team composition where as many heroes as possible must be included in a team up while maintaining a standard team structure (2-3 strategists, 1-2 vanguards, 1-2 duelists) to maximize the team's win rate.

using the win rate of mathups

we were able to find 23 different teams with 5 team ups, which is the maximum number of team ups a team can get.

Using the matchup winrate matrix, we get this result: 
...


a team should consist of 2 of each class, but when this is enforced, we see that there is no team with 2 of each class with 5 team ups.


In [1]:
import cvxpy as cp 
import pandas as pd
import warnings
warnings.filterwarnings("ignore")

In [10]:
team_ups = [
    ("Adam Warlock", "Star Lord"),
    ("Adam Warlock", "Mantis"),
    ("Thor", "Captain America"),
    ("Thor", "Storm"),
    ("Hela", "Loki"),
    ("Hela", "Thor"),
    ("Venom", "Spider Man"),
    ("Venom", "Peni Parker"),
    ("Hulk", "Doctor Strange"),
    ("Hulk", "Iron Man"),
    ("Rocket Raccoon", "The Punisher"),
    ("Rocket Raccoon", "Winter Soldier"),
    ("Invisible Woman", "The Thing"),
    ("Invisible Woman", "Mister Fantastic"),
    ("Magik", "Black Panther"),
    ("Magik", "Psylocke"),
    ("Human Torch", "Storm"),
    ("Iron Fist", "Luna Snow"),
    ("Spider Man", "Squirrel Girl"),
    ("Scarlet Witch", "Magneto"),
    ("Luna Snow", "Namor"),
    ("Luna Snow", "Jeff The Land Shark"),
    ("Groot", "Rocket Raccoon"),
    ("Groot", "Jeff The Land Shark"),
    ("Hulk", "Wolverine"),
    ("Invisible Woman", "Human Torch"),
    ("The Thing", "Wolverine"),
    ("Cloak & Dagger", "Moon Knight"),
    ("Hawkeye", "Black Widow")
]

win_rate_matrix = pd.read_csv("MarvelRivals_WinRate_matrix.csv", index_col=0)

heroes = [
    "Adam Warlock", "Black Panther", "Black Widow", "Captain America", "Cloak & Dagger",
    "Doctor Strange", "Groot", "Hawkeye", "Hela", "Hulk", "Human Torch",
    "Invisible Woman", "Iron Fist", "Iron Man", "Jeff The Land Shark", "Loki",
    "Luna Snow", "Magik", "Magneto", "Mantis", "Mister Fantastic",
    "Moon Knight", "Namor", "Peni Parker", "Psylocke", "Rocket Raccoon",
    "Scarlet Witch", "Spider Man", "Squirrel Girl", "Star Lord", "Storm",
    "The Punisher", "The Thing", "Thor", "Venom", "Winter Soldier", "Wolverine"
]

vanguards = ["Captain America", "Doctor Strange", "Groot", "Hulk", "Magneto", "Peni Parker", "The Thing", "Thor", "Venom"]
duelists = ["Black Panther", "Black Widow", "Hawkeye", "Hela", "Human Torch", "Iron Fist", "Iron Man", "Magik", "Mister Fantastic",
            "Moon Knight", "Namor", "Psylocke", "Scarlet Witch", "Spider Man", "Squirrel Girl", "Star Lord", "Storm", "The Punisher", 
            "Winter Soldier","Wolverine"]
strategists = ["Adam Warlock", "Cloak & Dagger", "Invisible Woman", "Jeff The Land Shark", "Loki", "Luna Snow", "Mantis", "Rocket Raccoon"]

if len(vanguards) + len(duelists) + len(strategists) != len(heroes):
    raise ValueError("The number of heroes is not correct")

hero_ids = {hero: idx for idx, hero in enumerate(heroes)}

vanguard_ids = [hero_ids[hero] for hero in vanguards]
duelist_ids = [hero_ids[hero] for hero in duelists]
strategist_ids = [hero_ids[hero] for hero in strategists]

team_up_tuples = [(hero_ids[a], hero_ids[b]) for a, b in sorted(team_ups) if hero_ids[a] < hero_ids[b]]

hero_avg_win_rates = win_rate_matrix.mean(axis=1).values

def printsolution(problem, x, y):
    selected_heroes = [heroes[i] for i in range(len(heroes)) if x.value[i] > 0.5]

    selected_team_ups = [(heroes[i], heroes[j]) for (i, j) in team_up_tuples if y[(i, j)].value > 0.5]

    # Calculate the average win rate using hero names
    selected_hero_avg_win_rates = [
    hero_avg_win_rates[i] for i in range(len(heroes)) if x.value[i] > 0.5
]

# Print the individual averages
    print("\nSelected Heroes' Average Win Rates:")
    for hero, rate in zip(selected_heroes, selected_hero_avg_win_rates):
        print(f"- {hero}: {rate:.2f}%")

    average_win_rate = problem.objective.value

    print("\nSelected Heroes:")
    for hero in selected_heroes:
        print("-", hero, ": Vanguard" if hero in vanguards else "Duelist" if hero in duelists else "Strategist")

    print("\nSelected Team-Ups:")
    for duo in selected_team_ups:
        print("-", duo)

    print(f"\nAverage Win Rate of Selected Team-Ups: {average_win_rate/600:.2%}")
    


In [17]:
def solve_teamups(teamUpCount, classLimit = False):
    x = cp.Variable(len(heroes), boolean=True) 
    y = {t: cp.Variable(boolean=True) for t in team_up_tuples}

    objective = cp.Maximize(cp.sum(cp.multiply(x, hero_avg_win_rates)))
    
    constraints = [
        cp.sum(x) == 6,
        cp.sum([y[t] for t in team_up_tuples]) == teamUpCount
    ]
    if classLimit:
        constraints.extend([
        cp.sum(x[vanguard_ids]) >= 1,   
        cp.sum(x[vanguard_ids]) <= 2,
        
        cp.sum(x[duelist_ids]) >= 1,
        cp.sum(x[duelist_ids]) <= 2,
        
        cp.sum(x[strategist_ids]) <= 3   
        ])
    
    # boolean AND operation
    for (i, j) in team_up_tuples:
        constraints.append(y[(i, j)] <= x[i])
        constraints.append(y[(i, j)] <= x[j])
        constraints.append(y[(i, j)] >= x[i] + x[j] - 1)
    problem = cp.Problem(objective, constraints)
    problem.solve()
    print(problem.status)
    if problem.status == cp.OPTIMAL:
        printsolution(problem, x, y)

In [12]:
solve_teamups(5, classLimit=False)

optimal

Selected Heroes' Average Win Rates:
- Hulk: 52.22%
- Invisible Woman: 47.21%
- Iron Man: 48.98%
- Mister Fantastic: 49.50%
- The Thing: 45.46%
- Wolverine: 54.37%

Selected Heroes:
- Hulk : Vanguard
- Invisible Woman Strategist
- Iron Man Duelist
- Mister Fantastic Duelist
- The Thing : Vanguard
- Wolverine Duelist

Selected Team-Ups:
- ('Hulk', 'Iron Man')
- ('Hulk', 'Wolverine')
- ('Invisible Woman', 'Mister Fantastic')
- ('Invisible Woman', 'The Thing')
- ('The Thing', 'Wolverine')

Average Win Rate of Selected Team-Ups: 49.62%


In [18]:
# no solution
solve_teamups(5, classLimit=True)

infeasible


In [14]:
solve_teamups(4, classLimit=False)


optimal

Selected Heroes' Average Win Rates:
- Groot: 50.94%
- Hulk: 52.22%
- Jeff The Land Shark: 51.85%
- Rocket Raccoon: 55.50%
- Winter Soldier: 53.12%
- Wolverine: 54.37%

Selected Heroes:
- Groot : Vanguard
- Hulk : Vanguard
- Jeff The Land Shark Strategist
- Rocket Raccoon Strategist
- Winter Soldier Duelist
- Wolverine Duelist

Selected Team-Ups:
- ('Groot', 'Jeff The Land Shark')
- ('Groot', 'Rocket Raccoon')
- ('Hulk', 'Wolverine')
- ('Rocket Raccoon', 'Winter Soldier')

Average Win Rate of Selected Team-Ups: 53.00%


In [15]:
solve_teamups(4, classLimit=True)

optimal

Selected Heroes' Average Win Rates:
- Groot: 50.94%
- Hulk: 52.22%
- Jeff The Land Shark: 51.85%
- Rocket Raccoon: 55.50%
- Winter Soldier: 53.12%
- Wolverine: 54.37%

Selected Heroes:
- Groot : Vanguard
- Hulk : Vanguard
- Jeff The Land Shark Strategist
- Rocket Raccoon Strategist
- Winter Soldier Duelist
- Wolverine Duelist

Selected Team-Ups:
- ('Groot', 'Jeff The Land Shark')
- ('Groot', 'Rocket Raccoon')
- ('Hulk', 'Wolverine')
- ('Rocket Raccoon', 'Winter Soldier')

Average Win Rate of Selected Team-Ups: 53.00%
